In [1]:
#%pip install country_converter
#%pip install psycopg2
#%pip install pyspark


In [2]:
#IMPORTS

import json, csv, urllib.request, requests
import country_converter as coco
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
from pyspark.sql.functions import rtrim 
from pyspark.sql import Row

In [3]:
#JSON Data - Covid19

url = "https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/json/"
response = urllib.request.urlopen(url)
dataCovid = json.loads(response.read())

In [4]:
#Dataframe - Covid19

covidDF = pd.DataFrame.from_dict(dataCovid)

In [5]:
#Field values convertion from 'country' 

covidDF['country'] = coco.convert(names=covidDF['country'], to='name_short')


Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total) not found in regex
Africa (total)

,country,country_code,continent,population,indicator,weekly_count,year_week,cumulative_count,source,rate_14_day,note
0,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-01,0,Epidemic intelligence national data,NaN,NaN
1,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-02,0,Epidemic intelligence national data,0,NaN
2,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-03,0,Epidemic intelligence national data,0,NaN
3,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-04,0,Epidemic intelligence national data,0,NaN
4,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-05,0,Epidemic intelligence national data,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
47161,Zimbabwe,ZWE,Africa,14862927,deaths,12.0,2022-06,5374,Epidemic intelligence national data,2.48941544286667,NaN
47162,Zimbabwe,ZWE,Africa,14862927,deaths,12.0,2022-07,5386,Epidemic intelligence national data,1.61475596294054,NaN
47163,Zimbabwe,ZWE,Africa,14862927,deaths,8.0,2022-08,5394,Epidemic intelligence national data,1.34562996911712,NaN
47164,Zimbabwe,ZWE,Africa,14862927,deaths,5.0,2022-09,5399,Epidemic intelligence national data,0.874659479926128,NaN


In [11]:
covidDF.head()

,country,country_code,continent,population,indicator,weekly_count,year_week,cumulative_count,source,rate_14_day,note
0,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-01,0,Epidemic intelligence national data,NaN,NaN
1,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-02,0,Epidemic intelligence national data,0,NaN
2,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-03,0,Epidemic intelligence national data,0,NaN
3,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-04,0,Epidemic intelligence national data,0,NaN
4,Afghanistan,AFG,Asia,38928341,cases,0.0,2020-05,0,Epidemic intelligence national data,0,NaN


In [12]:
#Convert columns name types from double to str (before Spark convertion)

all_columns = list(covidDF)
covidDF[all_columns] = covidDF[all_columns].astype(str)

In [13]:
#Convert Dataframe from Pandas to Spark

covidDF=spark.createDataFrame(covidDF) 
covidDF.printSchema()
covidDF.show()

root
 |-- country: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- population: string (nullable = true)
 |-- indicator: string (nullable = true)
 |-- weekly_count: string (nullable = true)
 |-- year_week: string (nullable = true)
 |-- cumulative_count: string (nullable = true)
 |-- source: string (nullable = true)
 |-- rate_14_day: string (nullable = true)
 |-- note: string (nullable = true)



2022-03-23 16:38:02,403 WARN scheduler.TaskSetManager: Stage 0 contains a task of very large size (5703 KiB). The maximum recommended task size is 1000 KiB.


+-----------+------------+---------+----------+---------+------------+---------+----------------+--------------------+-------------------+----+
|    country|country_code|continent|population|indicator|weekly_count|year_week|cumulative_count|              source|        rate_14_day|note|
+-----------+------------+---------+----------+---------+------------+---------+----------------+--------------------+-------------------+----+
|Afghanistan|         AFG|     Asia|  38928341|    cases|         0.0|  2020-01|               0|Epidemic intellig...|                nan| nan|
|Afghanistan|         AFG|     Asia|  38928341|    cases|         0.0|  2020-02|               0|Epidemic intellig...|                  0| nan|
|Afghanistan|         AFG|     Asia|  38928341|    cases|         0.0|  2020-03|               0|Epidemic intellig...|                  0| nan|
|Afghanistan|         AFG|     Asia|  38928341|    cases|         0.0|  2020-04|               0|Epidemic intellig...|                  

In [16]:
# COUNTRY DATAFRAME

# File location and type
file_location = "D:\countries_of_the_world.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
countriesDF = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(countriesDF)

IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: D:%5Ccountries_of_the_world.csv